In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [9]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [10]:
#Create dummy variables
geography=pd.get_dummies(dataset["Geography"],drop_first=True)
gender=pd.get_dummies(dataset['Gender'],drop_first=True)

In [11]:
## Concatenate the Data Frames
X=pd.concat([X,geography,gender],axis=1)

In [12]:
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [19]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch

In [10]:
#!pip install keras-tuner

In [22]:
# Data preprocessing
# Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [23]:
# Define the model-building function
def build_model(hp):
    model = Sequential()

    # Tune the number of units in the first Dense layer
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), input_dim=X_train.shape[1], activation='relu'))

    # Tune the number of layers and units in the hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu'))

    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

    # Tune the learning rate
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [24]:
# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of hyperparameter combinations to try
    directory='my_tuning_directory',  # Directory to store tuning logs and checkpoints
    project_name='my_first_tuning_project'  # Name for the tuning project
)

In [25]:
# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Perform the hyperparameter search
tuner.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)

Trial 10 Complete [00h 00m 08s]
val_accuracy: 0.8650000095367432

Best val_accuracy So Far: 0.8679999709129333
Total elapsed time: 00h 01m 18s


In [ ]:
{"trial_id": "00", "hyperparameters": {"space": [{"class_name": "Int", "config": {"name": "units", "default": null, "conditions": [], "min_value": 32, "max_value": 512, "step": 32, "sampling": "linear"}}, {"class_name": "Int", "config": {"name": "num_layers", "default": null, "conditions": [], "min_value": 1, "max_value": 3, "step": 1, "sampling": "linear"}}, {"class_name": "Int", "config": {"name": "units_0", "default": null, "conditions": [], "min_value": 32, "max_value": 512, "step": 32, "sampling": "linear"}}, {"class_name": "Choice", "config": {"name": "learning_rate", "default": 0.01, "conditions": [], "values": [0.01, 0.001, 0.0001], "ordered": true}}], "values": {"units": 128, "num_layers": 1, "units_0": 64, "learning_rate": 0.01}}, "metrics": {"metrics": {"loss": {"direction": "min", "observations": [{"value": [0.3960506021976471], "step": 0}]}, "accuracy": {"direction": "max", "observations": [{"value": [0.8387500047683716], "step": 0}]}, "val_loss": {"direction": "min", "observations": [{"value": [0.35267263650894165], "step": 0}]}, "val_accuracy": {"direction": "max", "observations": [{"value": [0.8634999990463257], "step": 0}]}}}, "score": 0.8634999990463257, "best_step": 0, "status": "COMPLETED", "message": null}

In [ ]:
{"trial_id": "09", "hyperparameters": {"space": [{"class_name": "Int", "config": {"name": "units", "default": null, "conditions": [], "min_value": 32, "max_value": 512, "step": 32, "sampling": "linear"}}, {"class_name": "Int", "config": {"name": "num_layers", "default": null, "conditions": [], "min_value": 1, "max_value": 3, "step": 1, "sampling": "linear"}}, {"class_name": "Int", "config": {"name": "units_0", "default": null, "conditions": [], "min_value": 32, "max_value": 512, "step": 32, "sampling": "linear"}}, {"class_name": "Choice", "config": {"name": "learning_rate", "default": 0.01, "conditions": [], "values": [0.01, 0.001, 0.0001], "ordered": true}}, {"class_name": "Int", "config": {"name": "units_1", "default": null, "conditions": [], "min_value": 32, "max_value": 512, "step": 32, "sampling": "linear"}}, {"class_name": "Int", "config": {"name": "units_2", "default": null, "conditions": [], "min_value": 32, "max_value": 512, "step": 32, "sampling": "linear"}}], "values": {"units": 96, "num_layers": 3, "units_0": 160, "learning_rate": 0.0001, "units_1": 32, "units_2": 64}}, "metrics": {"metrics": {"loss": {"direction": "min", "observations": [{"value": [0.3304722011089325], "step": 10}]}, "accuracy": {"direction": "max", "observations": [{"value": [0.8613749742507935], "step": 10}]}, "val_loss": {"direction": "min", "observations": [{"value": [0.33983755111694336], "step": 10}]}, "val_accuracy": {"direction": "max", "observations": [{"value": [0.8650000095367432], "step": 10}]}}}, "score": 0.8650000095367432, "best_step": 10, "status": "COMPLETED", "message": null}